<div style='background-color : orange'>
<a id='TableOfContents'></a>
    <b><u><i><h1 style='text-align : center'>
        Table of Contents
    </h1></i></u></b>
<li><a href='#imports'>Imports</a></li>
<li><a href='#q1'>Question 1</a></li>
<li><a href='#q2'>Question 2</a></li>
<li><a href='#q3'>Question 3</a></li>
<li><a href='#q4'>Question 4</a></li>
<li><a href='#misc'>Miscellaneous</a></li>

<div style='background-color : orange'>
<a id='imports'></a>
    <b><u><i><h1 style='text-align : center'>
        Imports
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [2]:
# Vectorization and dataframing
import numpy as np
import pandas as pd

# Visualizations
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# spark
import pyspark
from pyspark.sql import functions as F

# Play data
from pydataset import data

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Set default plt style to 'bmh'
mpl.style.use('bmh')

In [4]:
# Initialize a session
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/21 13:51:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<div style='background-color : orange'>
<a id='q1'></a>
    <b><u><i><h1 style='text-align : center'>
        Question 1
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>

<b>Question 1.</b> Create a spark data frame that contains your favorite programming languages.
- The name of the column should be <mark>language</mark>
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [5]:
# Create pandas dataframe of coding languages
languages = {
    'stuff' : ['python', 
                  'html', 
                  'java', 
                  'c++', 
                  'php', 
                  'sql', 
                  'other', 
                  'meh']
}
lang_df = pd.DataFrame(languages)
lang_df

,stuff
0,python
1,html
2,java
3,c++
4,php
5,sql
6,other
7,meh


In [6]:
# Create spark session of above pandas dataframe
lang = spark.createDataFrame(lang_df)
lang.show()

+------+
| stuff|
+------+
|python|
|  html|
|  java|
|   c++|
|   php|
|   sql|
| other|
|   meh|
+------+



In [7]:
# Rename 'stuff' column to 'language'
lang = lang.withColumnRenamed('stuff', 'language')
lang.show()

+--------+
|language|
+--------+
|  python|
|    html|
|    java|
|     c++|
|     php|
|     sql|
|   other|
|     meh|
+--------+



---

In [8]:
# Get the schema
lang.printSchema()

root
 |-- language: string (nullable = true)



---

In [9]:
# Get shape of spark dataframe
lang.count(), len(lang.columns)

(8, 1)

---

In [10]:
# Get first 5 records
lang.show(5)

+--------+
|language|
+--------+
|  python|
|    html|
|    java|
|     c++|
|     php|
+--------+
only showing top 5 rows



<div style='background-color : orange'>
<a id='q2'></a>
    <b><u><i><h1 style='text-align : center'>
        Question 2
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>
<li><a href='#q2a'>Question 2a</a></li>
<li><a href='#q2b'>Question 2b</a></li>

<b>Question 2.</b> Load the <mark>mpg</mark> dataset as a spark dataframe.

In [11]:
# Create spark session of 'mpg' dataset
mpg = spark.createDataFrame(data('mpg'))
mpg.show(2)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 2 rows



---

<a id='q2a'></a>
<b>Question 2a.</b> Create 1 column of output that contains a message like the one below:
- <mark>The 1999 audi a4 has a 4 cylinder engine.</mark>

For each vehicle.

<li><a href='#q2'>Question 2 Top</a></li>

In [12]:
# Identify where pertinent information lies
mpg.columns

['manufacturer',
 'model',
 'displ',
 'year',
 'cyl',
 'trans',
 'drv',
 'cty',
 'hwy',
 'fl',
 'class']

In [13]:
# Create statement column matching the request
mpg.select(
    (F.concat(
        F.lit('The '),
        mpg.year,
        F.lit(' '),
        mpg.manufacturer,
        F.lit(' '),
        mpg.model,
        F.lit(' has a '),
        mpg.cyl,
        F.lit(' cylinder engine')
    )).alias('statement')).show(5, truncate=False)

+----------------------------------------+
|statement                               |
+----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 6 cylinder engine|
+----------------------------------------+
only showing top 5 rows



---

<a id='q2b'></a>
<b>Question 2b.</b> Transform the <mark>trans</mark> column so that it only contains either <mark>manual</mark> or <mark>auto</mark>.

<li><a href='#q2'>Question 2 Top</a></li>

In [14]:
# Similar to if/else statement
mpg.select(
    (
        F.when(
            mpg.trans.contains('auto'), 'auto'
            ).otherwise('manual')
    ).alias('trans_type')
).show()

+----------+
|trans_type|
+----------+
|      auto|
|    manual|
|    manual|
|      auto|
|      auto|
|    manual|
|      auto|
|    manual|
|      auto|
|    manual|
|      auto|
|      auto|
|    manual|
|      auto|
|    manual|
|      auto|
|      auto|
|      auto|
|      auto|
|      auto|
+----------+
only showing top 20 rows



<div style='background-color : orange'>
<a id='q3'></a>
    <b><u><i><h1 style='text-align : center'>
        Question 3
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>
<li><a href='#q3a'>Question 3a</a></li>
<li><a href='#q3b'>Question 3b</a></li>
<li><a href='#q3c'>Question 3c</a></li>

<b>Question 3.</b> Load the <mark>tips</mark> dataset as a spark dataframe.

In [15]:
# Create spark session with 'tips' dataset
tips = spark.createDataFrame(data('tips'))
tips.show(2)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
+----------+----+------+------+---+------+----+
only showing top 2 rows



---

<a id='q3a'></a>
<b>Question 3a.</b> What percentage of observations are smokers?

<li><a href='#q3'>Question 3 Top</a></li>

In [16]:
# Identify columns
tips.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [17]:
# Get percent that are smokers
tips.filter(tips.smoker == 'Yes').count() / tips.count()

0.38114754098360654

---

<a id='q3b'></a>
<b>Question 3b.</b> Create a column that contains the tip percentage

<li><a href='#q3'>Question 3 Top</a></li>

In [18]:
# Get the percentage of tip amount
tips.withColumn(
    'tip_percent',
    F.round((tips.tip / tips.total_bill), 2)
).select('tip', 'total_bill', 'tip_percent').show(5)

+----+----------+-----------+
| tip|total_bill|tip_percent|
+----+----------+-----------+
|1.01|     16.99|       0.06|
|1.66|     10.34|       0.16|
| 3.5|     21.01|       0.17|
|3.31|     23.68|       0.14|
|3.61|     24.59|       0.15|
+----+----------+-----------+
only showing top 5 rows



---

<a id='q3c'></a>
<b>Question 3c.</b> Calculate the average tip percentage for each combination of sex and smoker.

<li><a href='#q3'>Question 3 Top</a></li>

In [19]:
# Show the average percentage for each sex and smoker combo
tips.withColumn(
    'tip_percentage',
    F.round((tips.tip / tips.total_bill), 2)
).groupby('sex', 'smoker').agg(
    F.round(F.mean('tip_percentage'), 2).alias('avg_percentage')).show()

+------+------+--------------+
|   sex|smoker|avg_percentage|
+------+------+--------------+
|  Male|    No|          0.16|
|Female|    No|          0.16|
|  Male|   Yes|          0.15|
|Female|   Yes|          0.18|
+------+------+--------------+



<div style='background-color : orange'>
<a id='q4'></a>
    <b><u><i><h1 style='text-align : center'>
        Question 4
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>
<li><a href='#q4a'>Question 4a</a></li>
<li><a href='#q4b'>Question 4b</a></li>
<li><a href='#q4c'>Question 4c</a></li>
<li><a href='#q4d'>Question 4d</a></li>
<li><a href='#q4e'>Question 4e</a></li>
<li><a href='#q4f'>Question 4f</a></li>
<li><a href='#q4g'>Question 4g</a></li>

<b>Question 4.</b> Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [22]:
# Create spark session of weather dataset
from vega_datasets import data
weather = spark.createDataFrame(data.seattle_weather())
weather.columns

['date', 'precipitation', 'temp_max', 'temp_min', 'wind', 'weather']

In [24]:
weather.show(5, vertical=True)

-RECORD 0----------------------------
 date          | 2012-01-01 00:00:00 
 precipitation | 0.0                 
 temp_max      | 12.8                
 temp_min      | 5.0                 
 wind          | 4.7                 
 weather       | drizzle             
-RECORD 1----------------------------
 date          | 2012-01-02 00:00:00 
 precipitation | 10.9                
 temp_max      | 10.6                
 temp_min      | 2.8                 
 wind          | 4.5                 
 weather       | rain                
-RECORD 2----------------------------
 date          | 2012-01-03 00:00:00 
 precipitation | 0.8                 
 temp_max      | 11.7                
 temp_min      | 7.2                 
 wind          | 2.3                 
 weather       | rain                
-RECORD 3----------------------------
 date          | 2012-01-04 00:00:00 
 precipitation | 20.3                
 temp_max      | 12.2                
 temp_min      | 5.6                 
 wind       

---

<a id='q4a'></a>
<b>Question 4a.</b> Convert the temperatures to fahrenheit.

<li><a href='#q4'>Question 4 Top</a></li>

In [27]:
# C ==> F == (xC * 9/5) + 32 
weather = weather.withColumn('temp_min',
                  weather.temp_min * 1.8 + 32
                  ).withColumn('temp_max',
                              weather.temp_max * 1.8 + 32)

In [28]:
# Check the results
weather.show(5, vertical=True)

-RECORD 0----------------------------
 date          | 2012-01-01 00:00:00 
 precipitation | 0.0                 
 temp_max      | 55.040000000000006  
 temp_min      | 41.0                
 wind          | 4.7                 
 weather       | drizzle             
-RECORD 1----------------------------
 date          | 2012-01-02 00:00:00 
 precipitation | 10.9                
 temp_max      | 51.08               
 temp_min      | 37.04               
 wind          | 4.5                 
 weather       | rain                
-RECORD 2----------------------------
 date          | 2012-01-03 00:00:00 
 precipitation | 0.8                 
 temp_max      | 53.06               
 temp_min      | 44.96               
 wind          | 2.3                 
 weather       | rain                
-RECORD 3----------------------------
 date          | 2012-01-04 00:00:00 
 precipitation | 20.3                
 temp_max      | 53.96               
 temp_min      | 42.08               
 wind       

---

<a id='q4b'></a>
<b>Question 4b.</b> Which month has the most rain, on average?

<li><a href='#q4'>Question 4 Top</a></li>

In [31]:
weather.groupby('weather').sum('precipitation').show()

+-------+------------------+
|weather|sum(precipitation)|
+-------+------------------+
|drizzle|               1.0|
|   rain|1321.8000000000006|
|    sun|239.39999999999995|
|   snow|             208.1|
|    fog|2655.7000000000003|
+-------+------------------+



In [35]:
weather.filter(weather.weather == 'rain'
              ).\
            withColumn(
                'month',
                F.month(weather.date)
            ).groupby('month').agg(
                (F.mean('precipitation')).alias('mean_precipitation'),
                (F.sum('precipitation')).alias('total_precipitation')
                ).sort('total_precipitation').show(2, vertical=True)

-RECORD 0----------------------------------
 month               | 9                   
 mean_precipitation  | 0.22499999999999998 
 total_precipitation | 0.8999999999999999  
-RECORD 1----------------------------------
 month               | 7                   
 mean_precipitation  | 1.8785714285714286  
 total_precipitation | 26.3                
only showing top 2 rows



---

<a id='q4c'></a>
<b>Question 4c.</b> Which year was the windiest?

<li><a href='#q4'>Question 4 Top</a></li>

In [32]:
weather.columns

['date', 'precipitation', 'temp_max', 'temp_min', 'wind', 'weather']

In [37]:
weather.withColumn(
    'year',
    F.year(weather.date)
).groupby(
'year'
).agg(
    (F.mean('wind')).alias('mean_wind'),
    (F.sum('wind')).alias('total_wind')
).sort(F.col('total_wind').desc()).first()

Row(year=2012, mean_wind=3.400819672131148, total_wind=1244.7)

---

<a id='q4d'></a>
<b>Question 4d.</b> What is the most frequent type of weather in January?

<li><a href='#q4'>Question 4 Top</a></li>

In [39]:
weather.filter(
    F.month(weather.date) == 1
).groupby('weather').count().sort(F.col('count').desc()).show()

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



---

<a id='q4e'></a>
<b>Question 4e.</b> What is the average high and low temperature on sunny days in July in 2013 and 2014?

<li><a href='#q4'>Question 4 Top</a></li>

In [40]:
weather.filter(
    (F.month(weather.date) == 7) &
    (F.year(weather.date).isin(['2013', '2014'])) &
    (weather.weather == 'sun')
).agg(
    F.round(F.mean('temp_max'), 2).alias('avg_high'),
    F.round(F.mean('temp_min'), 2).alias('avg_low')
).show()

+--------+-------+
|avg_high|avg_low|
+--------+-------+
|   80.29|  57.53|
+--------+-------+



---

<a id='q4f'></a>
<b>Question 4f.</b> What percentage of days were rainy in q3 of 2015?

<li><a href='#q4'>Question 4 Top</a></li>

In [42]:
weather.filter(
    F.quarter(weather.date) == 3).filter(
    F.year(weather.date) == 2015
).select(
    (F.when(weather.weather == 'rain', 1).otherwise(0)).alias('rain')
).agg((F.round(F.mean('rain'), 4)).alias('percentage_rain')).show()

+---------------+
|percentage_rain|
+---------------+
|         0.0217|
+---------------+



---

<a id='q4g'></a>
<b>Question 4g.</b> For each year, find what percentage of days it rained (had non-zero precipitation).

<li><a href='#q4'>Question 4 Top</a></li>

In [43]:
weather.withColumn(
    'year',
    F.year(weather.date)).withColumn(
    'has_rained',
    F.expr('precipitation > 0').cast('int')
).groupby('year').agg(
    F.round(F.avg('has_rained'), 4).alias('percentage_precipitation')
).show()

+----+------------------------+
|year|percentage_precipitation|
+----+------------------------+
|2012|                  0.4836|
|2013|                  0.4164|
|2014|                   0.411|
|2015|                  0.3945|
+----+------------------------+



<div style='background-color : orange'>
<a id='misc'></a>
    <b><u><i><h1 style='text-align : center'>
        Miscellaneous
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>